## Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)


## Player Count

* Display the total number of players


In [2]:
#Display the total number of players

df_totalplayer = pd.Series([len(purchase_data['SN'].value_counts())], name = 'Total Players')

df_totalplayer.to_frame()



,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
#Display the summary data frame of purchasing analysis

purchase_data['Item ID'].value_counts()
purchase_data['Price'].mean()
purchase_data['Item ID'].count()
purchase_data['Price'].sum()

df1 = pd.DataFrame([{'Number of Unique Items':len(purchase_data['Item ID'].value_counts()),
                           'Average Price':purchase_data['Price'].mean(),
                           'Number of Purchases':purchase_data['Item ID'].count(),
                           'Total Revenue':purchase_data['Price'].sum()}],index=[0])

df1['Average Price']=df1['Average Price'].astype(float).map('${:.2f}'.format)
df1['Total Revenue']=df1['Total Revenue'].astype(float).map('${:,.2f}'.format)

df_purchasinganalysis = df1.style.set_properties(**{'text-align':'left'})

display(df_purchasinganalysis)


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:

#Display the gender demographics, first filter gender and then obtain the unique value on SN

df_male = purchase_data.loc[purchase_data['Gender']=='Male',['SN','Gender']]
df_female = purchase_data.loc[purchase_data['Gender'] == 'Female',['SN','Gender']]

count_male = len(df_male['SN'].unique())
count_female = len(df_female['SN'].unique())
count_rest = len(purchase_data['SN'].value_counts()) - count_male - count_female

per_male = count_male/(count_male+count_female+count_rest)
per_female = count_female/(count_male+count_female+count_rest)
per_rest = count_rest/(count_male+count_female+count_rest)

data = {'Total Count':[count_male,count_female,count_rest],
        'Percentage of Players':[per_male,per_female,per_rest]}

df2 = pd.DataFrame(data,index=['Male','Female','Other / Non-Disclosed'])
df2['Percentage of Players'] = (df2['Percentage of Players']*100).astype(float).map('{:.2f}%'.format)

df_genderdemographics = df2.style.set_properties(**{'text-align':'left'})

display(df_genderdemographics)

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#Display the purchasing analysis (gender), basic idea is as like the above

df_male2 = purchase_data.loc[purchase_data['Gender']=='Male',['Purchase ID','Gender','Price']]
df_female2 = purchase_data.loc[purchase_data['Gender'] == 'Female',['Purchase ID','Gender','Price']]
df_rest2 = purchase_data.loc[purchase_data['Gender'] == 'Other / Non-Disclosed',['Purchase ID','Gender','Price']]

data2={'Purchase Count':[df_female2['Purchase ID'].count(),df_male2['Purchase ID'].count(),df_rest2['Purchase ID'].count()],
       'Average Purchase Price':[df_female2['Price'].mean(),df_male2['Price'].mean(),df_rest2['Price'].mean()],
       'Total Purchase Value':[df_female2['Price'].sum(),df_male2['Price'].sum(),df_rest2['Price'].sum()],
       'Avg Total Purchase per Person':[df_female2['Price'].sum()/count_female,
                                        df_male2['Price'].sum()/count_male,
                                        df_rest2['Price'].sum()/count_rest],
       'Gender':['Female','Male','Other / Non-Disclosed']}

df3 =pd.DataFrame(data2)

df3['Average Purchase Price'] = df3['Average Purchase Price'].map('${:.2f}'.format)
df3['Total Purchase Value'] = df3['Total Purchase Value'].map('${:.2f}'.format)
df3['Avg Total Purchase per Person'] = df3['Avg Total Purchase per Person'].map('${:.2f}'.format)

df_purchasinganalysis_gender = df3.set_index('Gender')

df_purchasinganalysis_gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#Display the age demographics table

df4 = purchase_data.sort_values(by=['SN']).groupby('SN').first().reset_index()

bins = [0,9,14,19,24,29,34,39,50]

age_group = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

df4['Age Ranges'] = pd.cut(df4['Age'],bins,labels=age_group)

df4a = df4[['Age','Age Ranges']]

df4b = df4a.groupby('Age Ranges')['Age'].count().reset_index()

df4b['Percentage of Players'] = round(df4b['Age']*100/df4b['Age'].sum(),2)

df4b.columns = ['Age Ranges','Total Count','Percentage of Players']

df4b['Percentage of Players']= df4b['Percentage of Players'].map('{}%'.format)

df_agedemographics = df4b.set_index('Age Ranges').rename_axis(None)

display(df_agedemographics)

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#Display purchasing analysis (age), applied a new method of .concat to merge multipul dataframe at once 
#and then remove duplicated columns. There is a 'SettingWithCopyWarning' as a 'chained assignment' operation involved

purchase_data['Age Ranges'] = pd.cut(purchase_data['Age'],bins,labels=age_group)

df5 = purchase_data[['Purchase ID','Price','Age Ranges']]

df5a= df5.groupby('Age Ranges')['Purchase ID'].count().reset_index()

df5b = df5.groupby('Age Ranges')['Price'].mean().reset_index()

ppl = df4b['Total Count']
df5c = df5.groupby('Age Ranges')['Price'].sum().reset_index()
df5c.columns = ['Age Ranges','Total Purchase Value']

df5d = pd.DataFrame(df5c['Age Ranges'])
df5d['Avg Total Purchase per Person'] = df5c['Total Purchase Value'].div(ppl,axis=0)

dataset = [df5a,df5b,df5c,df5d]

df5e = pd.concat(dataset, join='outer',axis=1)

df5f = df5e.loc[:,~df5e.columns.duplicated()]

df5f.columns = ['Age Ranges','Purchase Count','Average Purchase Price',
                'Total Purchase Value','Avg Total Purchase per Person']

df5f['Average Purchase Price'] = df5f['Average Purchase Price'].map('${:.2f}'.format)
df5f['Total Purchase Value'] = df5f['Total Purchase Value'].map('${:.2f}'.format)
df5f['Avg Total Purchase per Person'] = df5f['Avg Total Purchase per Person'].map('${:.2f}'.format)

df_purchasinganalysis_age = df5f.set_index('Age Ranges')

display(df_purchasinganalysis_age)

C:\Users\Matt\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Matt\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Matt\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#Display the top spenders, same method as age analysis

df6 = purchase_data[['Purchase ID','SN','Price']]

df6a = df6.groupby('SN')['Purchase ID'].count().reset_index()
df6b = df6.groupby('SN')['Price'].mean().reset_index()
df6c = df6.groupby('SN')['Price'].sum().reset_index()
df6c.columns = ['SN','Total']
dataset2 = [df6a,df6b,df6c]

df6d = pd.concat(dataset2, join='outer',axis=1)

df6e = df6d.loc[:,~df6d.columns.duplicated()]

df6e.columns = ['SN','Purchase Count','Average Purchase Price','Total Purchase Value']

df6f = df6e.sort_values(['Total Purchase Value'], ascending = False).head()

df6f['Average Purchase Price'] = df6f['Average Purchase Price'].map('${:.2f}'.format)
df6f['Total Purchase Value'] = df6f['Total Purchase Value'].map('${:.2f}'.format)

df_topspenders = df6f.set_index(['SN'])

df_topspenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
#Display the most popular items with same method above, this time applied 'merge' as the only difference

df7 = purchase_data[['Item ID','Item Name','Price']]

df7a = purchase_data.groupby(['Item ID','Item Name'])['Price'].count().reset_index()
df7b = purchase_data.groupby(['Item ID','Item Name'])['Price'].mean().reset_index()

df7c = purchase_data.groupby(['Item ID','Item Name'])['Price'].sum().reset_index()

df7d = pd.merge(df7a,df7b,on=['Item ID','Item Name'],how='outer')

df7e = pd.merge(df7d,df7c,on=['Item ID','Item Name'],how='outer')

df7e.columns = ['Item ID','Item Name','Purchase Count','Item Price','Total Purchase Value']

df7f = df7e.sort_values(['Purchase Count'], ascending = False).head()

df7f['Item Price'] = df7f['Item Price'].map('${:.2f}'.format)
df7f['Total Purchase Value'] = df7f['Total Purchase Value'].map('${:.2f}'.format)

df7_mostpopularitems = df7f.set_index(['Item ID','Item Name'])

df7_mostpopularitems

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
#Display the most profitable items, same method as above only change the sorting

df8 = df7e.sort_values(['Total Purchase Value'], ascending = False).head()

df8['Item Price'] = df8['Item Price'].map('${:.2f}'.format)
df8['Total Purchase Value'] = df8['Total Purchase Value'].map('${:.2f}'.format)

df_mostprofitableitems = df8.set_index(['Item ID','Item Name'])

df_mostprofitableitems

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


## Conclustion

a. It is clear to show that the number of male player is nearly 6 times of female players on this game.

b. Nearly half of the players are 20-24 years old and this group also has the 3rd high average spend per person.

c. The most popular items (most purchase counts) have higher value than the average price from the whole data, hence the game is having a positive price-setting strategy internally.